In [16]:
! pip install -U matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 20.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.7/229.7 kB 8.1 MB/s eta 0:00:00
  Using cached kiwisolver-1.4.4-cp310-cp310-macosx_11_0_arm64.whl (63 kB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 kB 15.0 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

/Users/yujian/.pyenv/versions/3.10.1/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/yujian/.pyenv/versions/3.10.1/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefINS2_6SymIntEEENS2_8optionalINS2_10ScalarTypeEEENS6_INS2_6LayoutEEENS6_INS2_6DeviceEEENS6_IbEENS6_INS2_12MemoryFormatEEE
  Referenced from: /Users/yujian/.pyenv/versions/3.10.1/lib/python3.10/site-packages/torchvision/image.so
  Expected in: /Users/yujian/.pyenv/versions/3.10.1/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")


In [3]:
device = "cpu"
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, X):
        x = self.flatten(X)
        logits = self.linear_relu_stack(x)
        return logits

training_data = datasets.FashionMNIST(root="fashion_mnist", train=True, transform=transforms.ToTensor())
test_data = datasets.FashionMNIST(root="fashion_mnist", train=False, transform=transforms.ToTensor())

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

In [4]:
model = NeuralNetwork()

In [5]:
learning_rate = 1e-3
batch_size=64
epochs=5

In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [10]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # predictions
        pred = model(X)
        loss = loss_fn(pred, y)

        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch%100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"Current loss: {loss:>7f}, [{current:>5d}/{size:>5d}]")

In [13]:
def test_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error:\n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")
    return 100*correct

In [14]:
new_model = NeuralNetwork()

In [18]:
plot_x = [] # epoch
plot_y = [] # accuracy

In [24]:
def optimize_loop(epoch_list, accuracy_list, epochs):
    for t in range(epochs):
        print(f"Epoch {t+1}\n")
        train_loop(train_dataloader, new_model, loss_fn, optimizer = torch.optim.SGD(new_model.parameters(), lr=learning_rate))
        test_loop(test_dataloader, new_model, loss_fn, optimizer = torch.optim.SGD(new_model.parameters(), lr=learning_rate))
    print("Done!")

In [25]:
import matplotlib
optimize_loop(plot_x, plot_y, 25)

Epoch 1

Current loss: 2.307214, [    0/60000]
Current loss: 2.300723, [ 6400/60000]
Current loss: 2.281263, [12800/60000]
Current loss: 2.274702, [19200/60000]
Current loss: 2.265342, [25600/60000]
Current loss: 2.233254, [32000/60000]
Current loss: 2.243182, [38400/60000]
Current loss: 2.213175, [44800/60000]
Current loss: 2.205196, [51200/60000]
Current loss: 2.186853, [57600/60000]
Test Error:
 Accuracy: 40.6%, Avg loss: 2.179526

Epoch 2

Current loss: 2.187114, [    0/60000]
Current loss: 2.184573, [ 6400/60000]
Current loss: 2.128052, [12800/60000]
Current loss: 2.142701, [19200/60000]
Current loss: 2.108447, [25600/60000]
Current loss: 2.043516, [32000/60000]
Current loss: 2.071839, [38400/60000]
Current loss: 1.999627, [44800/60000]
Current loss: 1.996043, [51200/60000]
Current loss: 1.941233, [57600/60000]
Test Error:
 Accuracy: 54.5%, Avg loss: 1.934662

Epoch 3

Current loss: 1.962199, [    0/60000]
Current loss: 1.942914, [ 6400/60000]
Current loss: 1.824689, [12800/60000]

In [28]:
import matplotlib.pyplot as plt

In [30]:
plot_x

[]